#  Introduction à ElasticSearch




Pour essayer les exemples présents dans ce tutoriel : 

<a href="https://github.com/linogaliana/python-datascientist/blob/master/notebooks/course/manipulation/07_elastic.ipynb" class="github"><i class="fab fa-github"></i></a>
[![nbviewer](https://img.shields.io/badge/visualize-nbviewer-blue)](https://nbviewer.jupyter.org/github/linogaliana/python-datascientist/blob/master/notebooks/course/manipulation/07_elastic.ipynb)
[![Onyxia](https://img.shields.io/badge/SSPcloud-Tester%20via%20SSP--cloud-informational&color=yellow?logo=Python)](https://datalab.sspcloud.fr/launcher/inseefrlab-helm-charts-datascience/jupyter?onyxia.friendlyName=%C2%ABpython-datascientist%C2%BB&resources.requests.memory=%C2%AB4Gi%C2%BB)

# Introduction

## Réplication de ce chapitre

Ce chapitre est plus exigeant en termes d'infrastructures que les précédents.
Il nécessite un serveur Elastic. Les utilisateurs du
[SSP Cloud](datalab.sspcloud.fr/) pourront répliquer les exemples de ce cours
car cette technologie est disponible (que ce soit pour indexer une base ou
pour requêter une base existante).

## Cas d'usage

Ce notebook recense et propose d'appréhender quelques outils utilisés
pour l'étude "Disparités territoriales de consommation d’aliments gras, salés et sucrés", Lino Galiana, Milena Suarez Castillo, Lionel Wilner (en cours!)

> Combien de calories dans ma recette de cuisine de ce soir? Combien de calories dans mes courses de la semaine?

L'objectif est de reconstituer, à partir de libellés de produits, les caractéristiques nutritionnelles d'une recette.
Le problème est que les libellés des tickets de caisse ne sont pas des champs textuels très propres, ils contiennent, 
par exemple, beaucoup d'abbréviations, toutes n'étant pas évidentes. 

Voici par exemple une série de noms de produits qu'on va utiliser par la suite: 


In [ ]:
ticket = ['CROISSANTS X6 400G',
          'MAQUEREAUX MOUTAR.',
          'IGP OC SAUVIGNON B',
          'LAIT 1/2 ECRM UHT',
          '6 OEUFS FRAIS LOCA',
          'ANANAS C2',
          'L POMME FUDJI X6 CAL 75/80 1KG ENV',
          'PLT MIEL',
          'STELLA ARTOIS X6',
          'COTES DU LUBERON AIGUEBRUN 75C']

A ces produits, s'ajoutent les ingrédients suivants, issus de la
[recette du velouté de potiron et carottes de Marmiton](https://www.marmiton.org/recettes/recette_veloute-de-potiron-et-carottes_19009.aspx)
qui sera notre plat principal :


In [ ]:
ingredients = ['500 g de carottes',
 '2 pommes de terre',
 "1 gousse d'ail",
 '1/2 l de lait',
 '1/2 l de bouillon de volaille',
 "1 cuillère à soupe de huile d'olive",
 '1 kg de potiron',
 '1 oignon',
 '10 cl de crème liquide (facultatif)']

Essayer de récupérer par webscraping cette liste est un bon exercice pour réviser
les concepts [vus précedemment](#webscraping)


In [ ]:
libelles = ticket + ingredients

On part avec cette liste dans notre supermarché virtuel. L'objectif sera de trouver
une méthode permettant passer à l'échelle:
automatiser les traitements, effectuer des recherches efficaces, garder une certaine généralité et flexibilité. 

Ce chapitre montrera par l'exemple l'intérêt d'`Elastic` par rapport à une solution 
qui n'utiliserait que du Python

# Données utilisées

## Les bases offrant des informations nutritionnelles 

Pour un nombre restreint de produits, on pourrait bien-sûr chercher à
la main les caractéristiques des produits en utilisant les 
fonctionalités d'un moteur de recherche:

```r
knitr::include_graphics("fraise.png")
```

Cependant, cette approche serait très fastidieuse et 
nécessiterait de récuperer, à la main, chaque caractéristique
pour chaque produit. Ce n'est donc pas envisageable.

Les données disponibles sur Google viennent de l'[USDA](https://fdc.nal.usda.gov/),
l'équivalent américain de notre Ministère de l'Agriculture. 
Cependant, pour des recettes comportant des noms de produits français, ainsi que 
des produits potentiellement transformés, ce n'est pas très pratique d'utiliser
une base de données de produits agricoles en Français. Pour cette raison,
nous proposons d'utiliser les deux bases suivantes, qui servent de base au travail de
Galiana et al. (à venir)

* L'[OpenFoodFacts database](https://fr.openfoodfacts.org/) qui est une base française, 
collaborative de produits alimentaires. Issue d'un projet [Data4Good](https://dataforgood.fr/), il s'agit d'une 
alternative opensource et opendata à la base de données de l'application [Yuka](https://yuka.io/). 
* La table de composition nutritionnelle [Ciqual](https://ciqual.anses.fr) produite par l'Anses. Celle-ci
propose la composition nutritionnelle _moyenne_ des aliments les plus consommés en France. Il s'agit d'une base de données
enrichie par rapport à celle de l'USDA puisqu'elle ne se cantonne pas aux produits agricoles non transformés. 
Avec cette base, il ne s'agit pas de trouver un produit exact mais essayer de trouver un produit type proche du produit
dont on désire connaître les caractéristiques. 


```r
knitr::include_graphics("openfood.png")
```

## Import 

Quelques fonctions utiles sont regroupées dans le script `utils.py` et importées dans le notebook. La base OpenFood peut être récupérée en ligne (opération qui peut prendre un peu de temps, on passe ici par le stockage interne de la plateforme en spécifiant `from_latest=False`). La base ciqual, plus légère, est récupérée elle directement en ligne.


In [ ]:
import utils

```
## Error in py_call_impl(callable, dots$args, dots$keywords): ModuleNotFoundError: No module named 'elasticsearch'
## 
## Detailed traceback: 
##   File "<string>", line 1, in <module>
##   File "/__w/python-datascientist/python-datascientist/content/course/manipulation/utils.py", line 6, in <module>
##     from elasticsearch.helpers import bulk, parallel_bulk
```

In [ ]:
openfood = utils.import_openfood()

```
## Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'utils' is not defined
## 
## Detailed traceback: 
##   File "<string>", line 1, in <module>
```

In [ ]:
ciqual = utils.import_ciqual()

```
## Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'utils' is not defined
## 
## Detailed traceback: 
##   File "<string>", line 1, in <module>
```


In [ ]:
openfood.head()

```
## Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'openfood' is not defined
## 
## Detailed traceback: 
##   File "<string>", line 1, in <module>
```


In [ ]:
ciqual.head()

```
## Error in py_call_impl(callable, dots$args, dots$keywords): NameError: name 'ciqual' is not defined
## 
## Detailed traceback: 
##   File "<string>", line 1, in <module>
```

# ElasticSearch ? Mais ce n'est pas du Python ?!

## Qu'est-ce qu'Elastic ? 

ElasticSearch c'est un logiciel qui fournit un moteur de recherche installé sur
un serveur (ou une machine personnelle) qu'il est possible de requêter depuis un client
(une session `Python` par exemple). C'est un moteur de recherche 
très performant, puissant et flexible, extrêmement utilisé dans le domaine de la datascience
sur données textuelles. Un cas d'usage est par exemple de trouver,
dans un corpus de grande dimension
(plusieurs sites web, livres...), un certain texte en s'autorisant des termes voisins
(verbes conjugués, fautes de frappes...).  

Le principe est le même que celui d'un moteur de recherche du web comme Google. 
D'un côté, l'ensemble à parcourir est indexé (c'est-à-dire XXX) pour être en 
mesure de parcourir de manière efficace l'ensemble du corpus.
De l'autre côté, la phase de recherche permet de retrouver l'élément du corpus le
plus cohérent avec la requête de recherche. L'indexation consiste, par exemple,
à pré-définir des traitements des termes du corpus pour gagner en efficacité
lors de la phase de recherche. En effet, l'indexation est une opération peu fréquente
par rapport à la recherche. Pour cette dernière, l'efficacité est cruciale (un site web 
qui prend plusieurs secondes à interpréter une requête simple ne sera pas utilisé). Mais, pour
l'indexation, ceci est moins crucial. 

ElasticSearch propose une interface graphique nommée Kibana. Celle-ci est pratique
pour tester des requêtes et pour superviser le serveur Elastic. Cependant,
pour le passage à l'échelle, notamment pour mettre en lien une base indexée dans
Elastic avec une autre source de données, les API proposées par ElasticSearch
sont beaucoup plus pratiques. Ces API permettent de connecter une session `Python` (idem pour `R`)
à un serveur Elastic afin de communiquer avec lui (échanger des flux via une API REST). 

## ElasticSearch et Python

En `Python`, le package officiel est [`elasticsearch`](https://elasticsearch-py.readthedocs.io/en/v7.12.0/).
Ce dernier permet de configurer les paramètres pour interagir avec un serveur, indexer 
une ou plusieurs bases, envoyer de manière automatisée un ensemble de requêtes
au serveur, récupérer les résultats directement dans une session `Python`...

# Limites de la distance de Levenshtein

On appelle distance de Levenshtein entre deux chaînes de caractères le coût minimal (en nombre d'opérations) pour transformer la première en la seconde par
* substitution
* insertion
* suppression

In [ ]:
import rapidfuzz # "Rapid fuzzy string matching in Python and C++ using the Levenshtein Distance" soit l'équivalent plus rapide de la librarie fuzzywuzzy
[rapidfuzz.string_metric.levenshtein('salut','slut', weights =(1,1,1)), # Suppression 
 rapidfuzz.string_metric.levenshtein('salut','saalut', weights =(1,1,1)), # Addition 
 rapidfuzz.string_metric.levenshtein('salut','selut', weights =(1,1,1))] # Substitution

## On va chercher les produits ciqual les plus proches de nos libellés en terme de distance textuelle classique

On écrit une fonction qui prend en argument une liste de libellés d'intérêt et une liste de candidat au match et renvoie le libellé le plus proche

In [ ]:
import time

def matchLevenstein(libelles, candidates):
    matches = dict()
    
    start_time = time.time()

    for l in libelles:
        # Calcul de la distance de levenstein entre le libellé l et tous les candidats à l'appariemment ! 
        # Initialisation avec le premier candidat, le plus proche jusqu'à preuve du contraire
        closest = candidates[0]
        levmin = rapidfuzz.string_metric.levenshtein(l, closest)  
        for candidate in candidates[1:]:
            if rapidfuzz.string_metric.levenshtein(l, candidate) < levmin:
                # Si un candidat se trouve être plus proche, il prend la place 
                closest = candidate
                levmin = rapidfuzz.string_metric.levenshtein(l, candidate)
                # (rmq: les cas d'égalité sont ici fréquents.. on favorise les derniers de la liste)
        print(l, '-', closest)
        matches[l]=closest
    
    print(80*'-')
    print(f"Temps d'exécution total : {(time.time() - start_time):.2f} secondes ---")
    
    return matches
    

Quid du match dans nos bases de données?

In [ ]:
matches = dict()
matches['ciqual_raw'] = matchLevenstein(libelles,list(ciqual['alim_nom_fr']))

In [ ]:
matches['openfood_raw'] = matchLevenstein(libelles,list(openfood['product_name']))

Cette première étape naïve est décevante ! 

**On a négligé une étape importante: la normalisation (ou nettoyage des textes)**
* harmonisation de la casse, suppression des accents...
* suppressions des mots outils (e.g. ici on va d'abord négliger les quantités pour trouver la nature de l'aliment, en particulier pour Ciqual)
    
**Le temps de calcul n'est pas forcément acceptable**

**La distance textuelle choisie** n'est pas toujours pertinente

On nettoie les libellés en mobilisant des expressions régulières et un dictionnaire de mots outils. On peut adapter le nettoyage à la base, par exemple dans ciqual, la cuisson est souvent renseignée et bruite les appariemments. Plus de détails dans la formation python ["Analyse textuelle: introduction"](https://datalab.sspcloud.fr/my-lab/catalogue/inseefrlab-helm-charts-datascience/jupyter/deploiement?init.personnalInit=https://git.lab.sspcloud.fr/g6ginq/formation_text_mining_public/-/raw/master/installPy.sh&onyxia.friendlyName=Text_Mining_Python) disponible sur le datalab.

In [ ]:
from utils import clean_libelle

In [ ]:
# Après avoir harmonisé la casse et retiré les accents (voir utils.py)
stopWords = ['KG','CL','G','L','CRUE?S?', 'PREEMBALLEE?S?']
replace_regex = {r'[^A-Z]': ' ', r'\b[A-Z0-9]{1,2}?\b':' '} # Retirer tout les caractères qui ne sont pas des lettres (chiffres, ponctuations); Retirer les caractères isolés

In [ ]:
ciqual = clean_libelle(ciqual, yvar = 'alim_nom_fr', replace_regex = replace_regex, stopWords = stopWords)

In [ ]:
openfood = clean_libelle(openfood, yvar = 'product_name', replace_regex = replace_regex, stopWords = stopWords)

In [ ]:
import pandas as pd
libellesDF = pd.DataFrame(libelles, columns = ['libel'])
libellesDF = clean_libelle(libellesDF, yvar = 'libel', replace_regex = replace_regex, stopWords = stopWords)

In [ ]:
libellesDF

In [ ]:
openfood.sample(10)

In [ ]:
ciqual.sample(10)

Est-ce que c'est mieux? Pas encore parfait, mais on progresse sur les produits appariés! 

In [ ]:
matches['openfood_clean'] = matchLevenstein(libellesDF['libel_clean'],list(openfood['libel_clean']))

Pour le temps de calcul, c'est pas encore ça.

## Réduire les temps de recherche

Finalement, l'idéal serait de disposer d'un **moteur de recherche** adapté à notre besoin, contenant les produits candidats, que l'on pourrait interroger, rapide en lecture, capable de classer les echos renvoyés par pertinence, que l'on pourrait requêter de manière flexible (par exemple, on pourrait vouloir signaler qu'un echo nous intéresse seulement si la donnée calorique n'est pas manquante). On pourrait même vouloir qu'il effectue pour nous des prétraitements sur les données. 

**Important pour la suite !**: 

* Lancer un service Elastic en parallèle sur le datalab via ce lien: https://datalab.sspcloud.fr/launcher/inseefrlab-helm-charts-datascience/elastic 
En pratique, une fois lancé, pas besoin d'ouvrir ce service Elastic pour continuer à suivre. 

_NB pour aller plus loin: Le lancement du service a créé dans votre `NAMESPACE Kubernetes` (l'ensemble de tout vos services) un cluster elastic (vous n'avez droit qu'à un cluster par namespace/compte d'utilisateur). Votre service jupyter est associé au même namespace. Pas besoin de tout saisir pour la suite, seulement que cette architecture permet à tout ce beau monde de dialoguer._

Le service Elastic doit apparaître, au même titre que ce service de formation jupyter, dans vos services sur le datalab. Vous pouvez aussi vérifier que votre Jupyter sait dialoguer avec votre Elastic, qui est prêt à vous écouter:

In [ ]:
! kubectl get statefulset

Nous allons utiliser la librairie `python` `elasticsearch` pour dialoguer avec notre moteur de recherche elastic. Les instructions ci dessous indiquent comment établir la connection.

In [ ]:
from elasticsearch import Elasticsearch
HOST = 'elasticsearch-master'

def elastic():
    """Connection avec Elastic sur le data lab"""
    es = Elasticsearch([{'host': HOST, 'port': 9200}], http_compress=True,  timeout=200)
    return es

es = elastic()

Maintenant que la connection est établie, deux étapes nous attendent:

1. **Indexation** Envoyer les documents parmi lesquels on veut chercher des echos pertinents dans notre elastic. Un index est une collection de document. Nous pourrions en créer deux: un pour les produits ciqual, un pour les produits openfood
2. **Requête** Chercher les documents les plus pertinents suivant une recherche textuelle flexible. Nous allons rechercher les libellés de notre recette et de notre liste de course.

On crée donc nos deux index:

In [ ]:
if not es.indices.exists('openfood'):
    es.indices.create('openfood')
if not es.indices.exists('ciqual'):
    es.indices.create('ciqual')

Pour l'instant, nos index sont vides! Ils contiennent 0 documents.

In [ ]:
es.count(index = 'openfood')

Nous allons en rajouter quelques uns ! 

In [ ]:
es.create(index = 'openfood',  id = 1, body = {'product_name': 'Tarte noix de coco', 'product_name_clean': 'TARTE NOIX COCO'})
es.create(index = 'openfood',  id = 2, body = {'product_name': 'Noix de coco', 'product_name_clean': 'NOIX COCO'})
es.create(index = 'openfood',  id = 3, body = {'product_name': 'Beurre doux', 'product_name_clean': 'BEURRE DOUX'})

In [ ]:
es.count(index = 'openfood')

Faisons notre première recherche: cherchons des noix de pécan! 

In [ ]:
es.search(index = 'openfood', q = 'noix de pécan')

Intéressons nous aux `hits` (résultats pertinents, ou echos) : nous en avons 2, le score maximal parmi les hits est mentionné dans `max_score` et correspond à celui du deuxième document indexé. Elastic nous fournit ici un **score de pertinence** dans notre recherche d'information, et classe ainsi les documents renvoyés.

Ici nous utilisons la configuration par défaut. Mais comment est calculé ce score?? Demandons à Elastic de nous expliquer le score du document `2` dans la requête `"noix de pécan"`.

In [ ]:
es.explain(index = 'openfood', id = 2, q = 'noix de pécan')

## En déduire ici comment est calculée la pertinence

Elastic nous explique donc que le score 0.9400072 est le maximum entre deux sous-scores, 0.4991 et 0.9400072. Pour chacun de ces sous scores, le détail de son calcul est donné. Le premier sous-score n'a accordé un score que par rapport au premier mot (noix), tandis que le second a accordé un score sur la base des deux mots déjà connu dans les documents ("noix" et "de"). Il a ignoré pécan! Jusqu'à présent, ce terme n'est pas connu dans l'index. 

La pertinence d'un mot pour notre recherche est construite sur une variante de la TF-IDF, considérant qu'un terme est pertinent s'il est souvent présent dans le document (Term Frequency) alors qu'il est peu fréquent dans les autres document (inverse document frequency). Ici les notations des documents 1 et 2 sont très proches, la différence est du à des IDF plus faibles dans le document 1, qui est pénalisé pour être légérement plus long. 

Bref, tout ça est un peu lourd, mais assez efficace, en tout cas moins rudimentaire que les distances caractères à caractères pour ramener des echos pertinents.

## Par contre pour l'instant, Elastic n'a pas l'air de gérer les fautes de frappes!

Pas le droit à l'erreur dans la requête:

In [ ]:
es.search(index = 'openfood',q = 'TART NOI')

Cela s'explique par la représentation des champs ('product_name' par exemple) qu'Elastic a inferré, puisque nous n'avons rien spécifié, représentations qui conditionnent la façon dont les champs sont analysés pour calculer la pertinence. Par exemple, regardons la représentation du champ `product_name`

In [ ]:
es.indices.get_field_mapping(index = 'openfood', fields = 'product_name')

Elastic a compris qu'il s'agissait d'un champ textuel, par contre, le type est `keyword` n'autorisant donc pas des analyses approximatives. Pour qu'un echo remonte, un des termes doit matcher exactement. Dommage ! Mais c'est parcequ'on a utilisé le mapping par défaut. En réalité, il est assez simple de préciser un mapping plus riche, autorisant une analyse "fuzzy" ou "flou".

# Une meilleure spécification du mapping, ou de comment vont être compris et analysé nos champs textuels lors des recherches

**On peut spécifier la façon dont l'on souhaite analyser le texte.** Par exemple, on peut préciser que l'on souhaite enlever des stopwords, raciniser, analyser les termes via des n-grammes pour rendre la recherche plus robuste aux fautes de frappes... Pour une présentation plus complète, voir https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html   
**L'important à retenir à ce stade est la flexibilité de l'outil.** On fournit dans la suite un fichier `settings_OpenFood.json` qui permet de préciser que nous souhaitons par exemple que les matchs sur n-grammes participent au score. Les n-grammes sont des séquences de n caractères ou plus généralement n éléments qui s'enchaînent séquentiellement. Par exemple, NOI et OIX sont des tri-grammes de caractères dans NOIX. Comparer les n-grammes composant des libellés peut permettre d'avoir dans des comparaisons à fautes de frappe/abbréviations près. Cela fait aussi plus de comparaisons à opérer ! D'où également, l'intérêt d'Elastic, qui intégre facilement et efficacement ces comparaisons. 

On va préciser un peu le schéma de données qu'on souhaite _indexer_, et aussi préciser comment les différents champs seront _analysés_.

### Une indexation plus adaptée

In [ ]:
import json

if es.indices.exists('openfood'):
    es.indices.delete('openfood')

with open('settings_OpenFood.json') as f:
    mapping = json.load(f)
    
es.indices.create(index = "openfood", body = mapping)    

Maintenant, les champs textuels "product_name" et "product_name_clean" vont pouvoir être analysé aussi via leur n-grammes et après racinisation (et l'un n'exclut pas l'autre!)

In [ ]:
es.indices.get_field_mapping(index = 'openfood', fields = 'product_name')

La fonction suivante va vous faire gagner du temps: **c'est parti, on envoie toute notre base OpenFood pour pouvoir la requêter!** Parcequ'en rester à 3 documents entrés à la main, ce n'est pas sérieux.

Du coup ça prend quelques minutes ... mais c'est pour nous en faire gagner ensuite. Cette opération est faite une fois, pour préparer des requêtes potentiellement nombreuses!

In [ ]:
from utils import index_elastic
index_elastic(es =es, index_name = "openfood",setting_file = 'settings_OpenFood.json', df = openfood[['product_name',"libel_clean","energy_100g","nutriscore_score"]].drop_duplicates())

In [ ]:
es.count(index = 'openfood')

## Nos premières requêtes

Vérifions qu'on recupère quelques tartes aux noix même si l'on fait plein de fautes:

In [ ]:
es.search(index = 'openfood', q = 'TART NOI', size = 3)

Et c'est plutôt rapide non?

In [ ]:
def matchElastic(libelles):
    matches = dict()
    
    start_time = time.time()

    for l in libelles:
        response = es.search(index = 'openfood', q = l, size = 1)
        if len(response['hits']['hits'])>0:
            matches[l] = response['hits']['hits'][0]['_source']['libel_clean']
    print(80*'-')
    print(f"Temps d'exécution total : {(time.time() - start_time):.2f} secondes ---")
    
    return matches

In [ ]:
matches = matchElastic(libellesDF['libel_clean'])

Et voilà, on a un outil très rapide de requête (à noter que je dispose d'un elastic probablement configuré différemment du votre, les performances peuvent varier), maintenant **on peut préciser des requêtes plus sophistiquées!**

En fait on a pas nettoyé les champs pour rien! On veut maintenant que notre requête porte spécifiquement sur le champ "libel_clean" (et pas indifféremment sur tout les champs textuels du document), ou encore en filtrant les produits avec un bon nutriscore.. Par exemple, des huiles d'olive avec un bon nutri score? 

à vous de déchiffrer cette requête (QUERY DSL)! Vous pouvez aussi explorer les possibilités de requêtes via la [doc Elastic](https://www.elastic.co/guide/en/elasticsearch/reference/6.8/query-dsl.html) et vous entrainer à un écrire avec votre index tout neuf.


In [ ]:
body = '''
{
  "size": "1",
  "query": {
    "bool": {
      "should": [
        { "match": { "libel_clean":  { "query":  "HUILE OLIVE" , "boost" : 10}}},
        { "match": { "libel_clean.ngr":   "HUILE OLIVE" }}],
      "minimum_should_match": 1,
      "filter": [
      { 
            "range" : {
                "nutriscore_score" : {
                    "gte" : 10,
                    "lte" : 20
                    }
                    }
                    }
      ]
    }
  }
}
'''

In [ ]:
es.search(index = 'openfood', body = body)

Qu'a-t-on demandé ici? De renvoyer 1 et 1 seul echo (`"size":"1"`) et seulement si celui ci a:
* `"should"`: Au moins un (`"minimum_should_match":"1"`) des termes des deux champs `libel_clean` et `libel_clean.ngr` qui matche sur un terme de _HUILE OLIVE_, l'analyse (la définition du "terme") étant réalisé soit en tant que `text` ("libel_clean") soit en tant que n-gramme `ngr` ("libel_clean.ngr", une analyse que nous avons spécifié dans le mapping) 
* `"filter"`: Le champ `float` nutriscore_score doit être compris entre 10 et 20 ("filter").  

A noter :
1. Les clauses (`"should"`+`"minimum_should_match":"1"`) peuvent être remplacé par un `"must"`, auquel cas, l'echo doit obligatoirement matché sur chaque clause.
2. Préciser dans `"filter"` (plutôt que dans "`should`") une condition signifie que celle-ci ne participe pas au score de pertinence. 


**C'est pas tout ça, mais on a pas encore un appariemment très satisfaisant, en particulier sur les boissons**

In [ ]:
matches

## S'aider de dictionnaires, et les construire

Une méthode simple pour faire comprendre par exemple que "Stella Artois" est une marque de bière est de créer un dictionnaire de marques de bière. A partir de celui-ci, on pourra déterminer si oui ou non on a affaire à une bière. C'est typiquement de l'information connue, publique. On enrichie ainsi l'information disponible. 

Oui sauf que à la main, c'est pas très marrant!

Alors comment récupérer cela efficacement? Wikipedia offre un service web qui donne accès à ses contenus (à utiliser sans en abuser, voir les bonnes pratiques https://www.mediawiki.org/wiki/API:Etiquette). Cela peut être une idée pour constituer des listes de marques. 

In [ ]:
import urllib
import re

def dictionnary_from_wiki_category(categorie = ['Bière blonde','Vin_français','Marque de bière'], n: int = 10, filters = 'Utilisateur|Discussion|Classement|Liste|Catégorie', sub='\(.*\)'):
    if isinstance(categorie, list):
        categorie = categorie[0]
    url = "https://fr.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:" + urllib.parse.quote(categorie) + "&cmlimit=" + str(n) + "&format=json"
    with urllib.request.urlopen(url) as response:
        jsonresp = response.read()
    jsonresp = json.loads(jsonresp)['query']['categorymembers']
    if len(jsonresp)>1:
        res = [re.sub(sub,'',x['title']).strip() for x in jsonresp if not re.match(filters,x['title'])]
        return(res)
    else:
        return []

In [ ]:
dictionnary_from_wiki_category("Marque de bière", n = 20)

In [ ]:
dictionnary_from_wiki_category("Fromage français", n = 20)

**Et donc?** 

à ce stade et faute d'information supplémentaire, on peut associer à toutes les appellations de vins rouges, ou encore toutes les appelations de fromage non encore appariées, leur qualité nutritionnelle moyenne (https://ciqual.anses.fr/#/aliments/5210/vin-(aliment-moyen) ou https://ciqual.anses.fr/#/aliments/12999/fromage-(aliment-moyen)). Etant donné la diversité de ces classes dans le panier de consommation des français, cela peut valoir le coup.

Cela peut se faire en créant par exemple un index elastic à partir de ces listes de produits wikipédia (associé à une information nutritionnelle "moyenne") pour venir y requêter les produits du reliquat. 

# Considérer les embeddings de mots existants?

Elastic ramène des echos en général bien ordonnés par pertinence, très bien, mais typiquement, cela peut ratisser large, surtout quand on utile les n-grammes ! Et parfois, il n'y a tout simplement pas l'information dans Ciqual ou Openfood Facts. 
Il faut donc définir un critère pour décider de classer le premier echo comme un match, **soit une notion de proximité entre deux libellés**. Une possibilité est d'avoir recours aux distances textuelles classiques, dérivées de la distance de levenstein (avec par exemple une normalisation pour tenir compte de la longueur variable des libellés). 

Une autre option pour définir des distances entre les termes est de s'appuyer sur champ de recherche récent qui cherche à définir des "plongements de mots".

**Word Embeddings ou plongement de mots:** représentation vectorielle des chaînes de charactères, de mots, de phrases, de la langue en général, obtenu à partir de modèles de Deep Learning entraînés sur des corpus de texte importants. cf [l'introduction à l'analyse textuelle dans ce set de formation qui évoque Word2Vec](https://datalab.sspcloud.fr/my-lab/catalogue/inseefrlab-helm-charts-datascience/jupyter/deploiement?init.personnalInit=https://git.lab.sspcloud.fr/g6ginq/formation_text_mining_public/-/raw/master/installPy.sh&onyxia.friendlyName=Text_Mining_Python).


Des plongements de mots spécifiques à la langue française sont mis à dispositions par des équipes de recherches:  
https://camembert-model.fr/  
https://github.com/getalp/Flaubert   

Là aussi, on prend un peu de temps à installer les librairies nécessaires, et à charger les plongements de mots

In [ ]:
! pip install -q -q -q torch transformers

In [ ]:
import torch
from transformers import FlaubertModel, FlaubertTokenizer

modelname = 'flaubert/flaubert_base_cased' 

# Load pretrained model and tokenizer
flaubert, log = FlaubertModel.from_pretrained(modelname, output_loading_info=True)
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)

Calculons quelques distances entre quelques libellés bien choisis au sens de l'embedding "Flaubert"

In [ ]:
lib1 = "bière framboise"
lib2 = "barquette framboises"
lib3 = "bière blonde"

Représentation en `token` (un enchaînement de caractère = 1 entier l'indexant) `flaubert_tokenizer.encode`

In [ ]:
token1 = torch.tensor([flaubert_tokenizer.encode(lib1)])
token2 = torch.tensor([flaubert_tokenizer.encode(lib2)])
token3 = torch.tensor([flaubert_tokenizer.encode(lib3)])


Représentation vectorielle dense (moyenne des représentations vectorielles de chaque token), en passant les tokens au modèle `flaubert`

In [ ]:
vec1 = flaubert(token1)[0].mean(axis = 1)
vec2 = flaubert(token2)[0].mean(axis = 1)
vec3 = flaubert(token3)[0].mean(axis = 1)


In [ ]:
[torch.sum((vec1-vec2)**2),torch.sum((vec3-vec2)**2),torch.sum((vec1-vec3)**2)]

Les deux bières sont plus proches l'une de l'autre que les autres couples de libellés. Mais ça demande un peu plus d'exploration à ce stade, voire de retravailler cet embedding pour notre problème. A noter que ces embeddings ne sont pas vraiment pensés pour des libellés courts et bruités, plutôt pour des textes. 

Reentrainés à la marge pour notre cas d'usage (["tranfer learning"](https://blog.baamtu.com/en/word2vec-camembert-use-embedding-models/
)), en supposant que nous disposons d'un échantillon d'entrainement de "vrai" couples de libellés appariés), nous pourrions peut être disposer d'une distance textuelle moins rudimentaire que la distance de levenstein.

C'est en effet l'ambition des embeddings de mots de représenter la sémantique au delà de la proximité des chaînes de caractères.

# Conclusion

On a pas encore calculé notre apport total de calories sur la base de notre liste, plutôt exploré quelques idées pour traiter le problème. A vous de jouer maintenant avec tout ces ingrédients! Bon appétit
